In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "..."  # API 키 입력
os.environ["AWS_DEFAULT_REGION"] = "us-east-1" # Bedrock 지원하는 리전 입력

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 경로 설정
file_path = "./nke-10k-2023.pdf"

# PyPDFLoader를 사용하여 PDF 파일 로드
loader = PyPDFLoader(file_path)

# PDF 문서를 로드하고 각 페이지의 내용을 저장
docs = loader.load()

# 로드된 문서 수 확인
print(len(docs))

# 첫 번째 문서 내용 및 메타데이터 확인
print(docs[0].page_content[0:100])
print(docs[0].metadata)


106
FORM 10-K FORM 10-K
{'source': './nke-10k-2023.pdf', 'page': 0}


In [4]:
print(docs[1].page_content[0:100])

UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K 
(Mark One)
☑ ANNU


In [7]:
'''
chunk_size = 5, overlap = 2
ABCDEFGHIJK

ABCDE
DEFGH
GHIJK
'''


from langchain_aws import BedrockEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 스플리터 설정 (1000자 단위로 문서를 나누고, 중첩 200자 적용)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

위 오버랩과 청크사이즈를 기준으로 쪼개주어 501개가 됐음

In [8]:
len(splits)

501

In [9]:
# 벡터 스토어에 임베딩 적용 및 문서 저장
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=BedrockEmbeddings(
        model_id='amazon.titan-embed-text-v1',
    )
)

# 리트리버 생성
retriever = vectorstore.as_retriever()

In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock

# LLM 설정 (Claude 모델 사용)
llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
)

# 시스템 프롬프트 설정
system_prompt = (
    "당신은 질문에 답하는 어시스턴트입니다. "
    "다음 문서에서 제공된 내용을 사용하여 질문에 답변하세요. "
    "모르면 모른다고 말하세요. 최대 세 문장으로 간결하게 답하세요."
    "\\n\\n"
    "{context}"
)

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# RAG 체인 생성
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# 질문에 대한 응답 생성
results = rag_chain.invoke({"input": "나이키의 2023년 매출은 얼마인가요?"})

# 결과 출력
print(results['answer'])


나이키의 2023 회계연도 매출은 512억 달러입니다. 이는 2022 회계연도 대비 10% 증가한 수치이며, 환율 중립 기준으로는 16% 증가했습니다.


In [15]:
results

{'input': '나이키의 2023년 매출은 얼마인가요?',
 'context': [Document(id='bad0612c-3551-4076-ac22-91cb13949d57', metadata={'source': './nke-10k-2023.pdf', 'page': 36}, page_content='FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTS\nThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and \nmajor product line:\nFISCAL 2023 COMPARED TO FISCAL 2022\n•NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported \nand currency-neutral basis, respectively. The increase was due to higher revenues in North America, Europe, Middle East & \nAfrica ("EMEA"), APLA and Greater China, which contributed approximately 7, 6, 2 and 1 percentage points to NIKE, Inc. \nRevenues, respectively. \n•NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues,  increased  10% and 16% on a reported and \ncurrency-neutral basis, respectively. This increase was primarily due to higher revenues in Men

In [14]:
results["context"][0].page_content

'FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTS\nThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and \nmajor product line:\nFISCAL 2023 COMPARED TO FISCAL 2022\n•NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported \nand currency-neutral basis, respectively. The increase was due to higher revenues in North America, Europe, Middle East & \nAfrica ("EMEA"), APLA and Greater China, which contributed approximately 7, 6, 2 and 1 percentage points to NIKE, Inc. \nRevenues, respectively. \n•NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues,  increased  10% and 16% on a reported and \ncurrency-neutral basis, respectively. This increase was primarily due to higher revenues in Men\'s, the Jordan Brand, \nWomen\'s and Kids\' which grew 17%, 35%,11% and 10%, respectively, on a wholesale equivalent basis.'